# Puzzles mit C++ und Backtracking gelöst

die Puzzles gibts von https://www.smartgames.eu/de/spiele-für-einen-spieler/iq-puzzler-pro

In [1]:
#include <iostream>
#include "parts.h"
using namespace std;

In [2]:
void remove(part *part)
{
    auto variant = &part->variants[part->vIdx];
    for (int i = 0; i < variant->length; i++)
    {
        board[i + part->yPos] &= ~(variant->data[i] << part->xPos);
    }
    part->xPos = -1;
    placed--;
}


In [3]:
bool checkAndPlace(part *part, int x, int y, int variant)
{
    checkCount++;
    auto toCheck = &part->variants[variant];
    auto l = toCheck->length;
    auto d = toCheck->data;

    int *linePtr = &d[l];
    int *boardValue = &board[l] + y;

    while (l--)
    {
        linePtr--;
        boardValue--;
        int testValue = *linePtr;

        bool result = ((testValue ^ *boardValue >> x) & testValue) == testValue;

        if (!result)
            return false;
    }
    
    // Place
    part->xPos = x;
    part->yPos = y;
    part->vIdx = variant;
    placed++;

    auto variantPart = &part->variants[variant];
    for (int i = 0; i < variantPart->length; i++)
    {
        board[i + y] |= variantPart->data[i] << x;
    }
    return true;
}

In [4]:
bool solve()
{
    for (int p = 0; p < 12; p++)
    {
        auto currentPart = &parts[p];
        if (currentPart->xPos == -1)
        {
            for (int v = 0; v < currentPart->vCount; v++)
            {
                partChecks++;
                auto currentVariant = &currentPart->variants[v];

                for (int r = 0; r < 5 - currentVariant->length + 1; r++)
                {
                    for (int c = 0; c < 11 - currentVariant->width + 1; c++)
                    {
                        if (checkAndPlace(currentPart, c, r, v))
                        {
                            positiveCheck++;

                            if (placed == 12)
                            {
                                return true; // Finale Lösung gefunden
                            }

                            auto result = solve();
                            if (result)
                            {
                                return true;
                            }
                            else
                            {
                                remove(currentPart);
                            }
                        }
                    }
                }
            }
            return false; // das puzzle wurde nicht gefunden
        }
    }
    return false;
}


In [5]:
void clearBoard() {
    for (int i=0; i<5;i++) {
        board[i]=0;
    }
    for (int i=0;i<12;i++) {
        parts[i].xPos=-1;
    }
    placed=0;
    partChecks=0;
    checkCount=0;
    positiveCheck=0;
}

In [6]:
void init(puzzle p)
{
    clearBoard();
    for (int i = 0; i < p.puzzleCount; i++)
    {
        auto pu = p.puzzles[i];
        part *pa = findPart(pu.id);
        if (!checkAndPlace(pa, pu.xPos, pu.yPos, pu.vIdx))
        {
            std::cout << "Puzzle kann nicht plaziert werden" << pu.id << std::endl;
        }
    }
}


In [7]:
int runWith(puzzle dataSet)
{
    init(dataSet);

    std::cout << "starting..." << std::endl;
    showBoard();

    if (solve())
    {
        std::cout << "Die Lösung:" << std::endl;
    }
    else
    {
        std::cout << "Es gibt leider keine Lösung" << std::endl;
    }
    showBoard();

    std::cout
        << "Varianten getestet: " << partChecks << std::endl
        << "Puzzle Positionen getestet: " << checkCount << std::endl
        << "davon passend: " << positiveCheck << std::endl;

    return 0;
}

In [8]:
%timeit -n 1 -r 1 runWith(master[0]);

starting...
y y y y i i b b b . . 
. . y m m i . . b . . 
. . m m i i . . b . . 
. . m . . . . . . . . 
. . . . . . . . . . . 
Die Lösung:
y y y y i i b b b u u 
e e y m m i c c b p u 
e e m m i i c o b p u 
a e m r r o o o g p p 
a a a a r r o g g g p 
Varianten getestet: 33863
Puzzle Positionen getestet: 1117641
davon passend: 6089
364 ms +- 0 ns per loop (mean +- std. dev. of 1 run, 1 loop each)


In [9]:
%timeit -n 1 -r 1 runWith(starter[0]);


starting...
p y y y y o u u u . . 
p p g y o o u . . . . 
b p g g r o o . . . . 
b p g c r r e e . . . 
b b b c c r e e e . . 
Die Lösung:
p y y y y o u u u a a 
p p g y o o u i i i a 
b p g g r o o i m i a 
b p g c r r e e m m a 
b b b c c r e e e m m 
Varianten getestet: 52
Puzzle Positionen getestet: 1525
davon passend: 11
1.33 ms +- 0 ns per loop (mean +- std. dev. of 1 run, 1 loop each)
